In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
from sklearn.linear_model import LinearRegression

In [4]:
df = sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [5]:
df = df[['total_bill', 'tip']]
df.head(3)

,total_bill,tip
0,16.99,1.01
1,10.34,1.66
2,21.01,3.50


- EDA part is left to do just to show the ML flow

In [6]:
from sklearn.model_selection import train_test_split

# independent and dependent feature split
X = df.drop('tip', axis=1)
y = df['tip']
type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [7]:
# train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=42
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((195, 1), (49, 1), (195,), (49,))

In [8]:
lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

LinearRegression()

In [16]:
X_test.loc[24]

total_bill    19.82
Name: 24, dtype: float64

In [15]:
lr_model.predict([X_test.loc[24]])

c:\Users\rajba\.conda\envs\tech-axis\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([3.04525623])

In [17]:
lr_model.predict([[19.82]])

c:\Users\rajba\.conda\envs\tech-axis\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([3.04525623])

In [9]:
predictions = lr_model.predict(X_test)
predictions

array([3.04525623, 1.86330727, 3.55119456, 3.69452593, 2.31576375,
       2.83881627, 3.96728338, 2.26014262, 2.50615915, 2.57033737,
       2.88160176, 2.07723468, 2.06439904, 2.47407003, 2.00236009,
       2.91903905, 2.92652651, 3.23351235, 2.68478854, 5.33107064,
       3.13831465, 3.13403611, 2.4558862 , 1.94673896, 3.16077703,
       2.17564129, 2.02375283, 3.62927807, 2.68906708, 6.07767732,
       4.99734388, 1.75313465, 2.83025918, 3.09552917, 2.74040966,
       3.50092162, 2.21200895, 5.53644096, 2.33287794, 3.35010279,
       2.04942412, 2.47834858, 3.48701634, 2.03017065, 2.03124029,
       1.25361414, 2.05798121, 2.92438724, 1.73388118])

In [18]:
test = pd.DataFrame({
    'total_bill': X_test['total_bill'],
    'tip': y_test,
    'predictions': predictions
})
test.head()

,total_bill,tip,predictions
24,19.82,3.18,3.045256
6,8.77,2.00,1.863307
153,24.55,2.00,3.551195
211,25.89,5.16,3.694526
198,13.00,2.00,2.315764


In [20]:
test['diff'] = test['predictions'] - test['tip']
test.head()

,total_bill,tip,predictions,diff
24,19.82,3.18,3.045256,-0.134744
6,8.77,2.00,1.863307,-0.136693
153,24.55,2.00,3.551195,1.551195
211,25.89,5.16,3.694526,-1.465474
198,13.00,2.00,2.315764,0.315764


In [23]:
from sklearn.metrics import root_mean_squared_error, r2_score

root_mean_squared_error(y_test, predictions)

0.7541977545199626

In [26]:
import math
math.sqrt(500)

22.360679774997898

In [27]:
r2_score(y_test, predictions)

0.5449381659234663

In [ ]:
class LinearRegressionCustom:
    def __init__(self, learning_rate=0.001, epoch=100, bias=True) -> None:
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.bias = bias